A Monte Carlo simulation is a model used to predict the probability of different outcomes when the intervention of random variables is present.

Monte Carlo simulations help to explain the impact of risk and uncertainty in prediction and forecasting models.

The basis of a Monte Carlo simulation involves assigning multiple values to an uncertain variable to achieve multiple results and then averaging the results to obtain an estimate.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style('whitegrid')

In [2]:
avg = 1
std_dev = .1
num_reps = 500
num_simulations = 1000

In [3]:
pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

In [5]:
sales_target_values = [75_000, 100_000, 200_000, 300_000, 400_000, 500_000]
sales_target_prob = [.3, .3, .2, .1, .05, .05]
sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)

In [7]:
df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                               'Sales_Target': sales_target})

df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

df.head()

,Pct_To_Target,Sales_Target,Sales
0,1.03,100000,103000.0
1,0.92,100000,92000.0
2,0.96,300000,288000.0
3,0.90,100000,90000.0
4,0.93,100000,93000.0


In [12]:
def calc_commission_rate(x):
    """ Return the commission rate based on the table:
    0-90% = 2%
    91-99% = 3%
    >= 100 = 4%
    """
    if x <= .90:
        return .02
    if x <= .99:
        return .03
    else:
        return .04

In [13]:
df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

In [14]:
df.head()

,Pct_To_Target,Sales_Target,Sales,Commission_Rate,Commission_Amount
0,1.03,100000,103000.0,0.04,4120.0
1,0.92,100000,92000.0,0.03,2760.0
2,0.96,300000,288000.0,0.03,8640.0
3,0.90,100000,90000.0,0.02,1800.0
4,0.93,100000,93000.0,0.03,2790.0


In [15]:
# Define a list to keep all the results from each simulation that we want to analyze
all_stats = []

# Loop through many simulations
for i in range(num_simulations):

    # Choose random inputs for the sales targets and percent to target
    sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)
    pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

    # Build the dataframe based on the inputs and number of reps
    df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                                   'Sales_Target': sales_target})

    # Back into the sales number using the percent to target rate
    df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

    # Determine the commissions rate and calculate it
    df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
    df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

    # We want to track sales,commission amounts and sales targets over all the simulations
    all_stats.append([df['Sales'].sum().round(0),
                      df['Commission_Amount'].sum().round(0),
                      df['Sales_Target'].sum().round(0)])

In [17]:
results_df = pd.DataFrame.from_records(all_stats, columns=['Sales',
                                                           'Commission_Amount',
                                                           'Sales_Target'])

In [29]:
results_df.describe().round(2).style.format('{:,}')

,Sales,Commission_Amount,Sales_Target
count,"1,000.0","1,000.0","1,000.0"
mean,"83,753,728.25","2,860,228.68","83,739,600.0"
std,"2,692,891.32","100,456.56","2,687,560.43"
min,"76,297,500.0","2,540,852.0","76,400,000.0"
25%,"81,833,500.0","2,789,403.5","81,825,000.0"
50%,"83,723,125.0","2,858,515.0","83,750,000.0"
75%,"85,462,687.5","2,931,596.5","85,400,000.0"
max,"91,882,500.0","3,200,952.0","91,350,000.0"


In [30]:
results_df.describe().round(2)

,Sales,Commission_Amount,Sales_Target
count,1000.00,1000.00,1000.00
mean,83753728.25,2860228.68,83739600.00
std,2692891.32,100456.56,2687560.43
min,76297500.00,2540852.00,76400000.00
25%,81833500.00,2789403.50,81825000.00
50%,83723125.00,2858515.00,83750000.00
75%,85462687.50,2931596.50,85400000.00
max,91882500.00,3200952.00,91350000.00
